# Implémentez un modèle de scoring

*Notebook modélisation*

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

**Mission :**

- Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

- Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

- Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

- Mettre en œuvre une approche globale MLOps de bout en bout, du tracking des expérimentations à l’analyse en production du data drift.
    + Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations
    + Lancer l’interface web 'UI MLFlow" d'affichage des résultats du tracking
    + Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”
    + Tester le serving MLFlow
    + Gérer le code avec le logiciel de version Git
    + Partager le code sur Github pour assurer une intégration continue
    + Utiliser Github Actions pour le déploiement continu et automatisé du code de l’API sur le cloud
    + Concevoir des tests unitaires avec Pytest (ou Unittest) et les exécuter de manière automatisée lors du build réalisé par Github Actions

## 1 - Préparer l'environnement d'expérimentation

### 1.1 - Imports

In [4]:
import mlflow
import mlflow.tracking
from mlflow.models import infer_signature

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder

from contextlib import contextmanager
import time

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
print("seaborn version", sns.__version__)
print("mlflow version", mlflow.__version__)
print("sklearn version", sklearn.__version__)

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 200
pd.options.display.max_columns = 130

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0
seaborn version 0.13.2
mlflow version 2.20.1
sklearn version 1.6.1


### 1.2 - Lancement du serveur MLFlow

Dans le terminal de Powershell Prompt d'Anaconda, lancer cette instruction :

mlflow server --host 127.0.0.1 --port 5000 --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

Cette commande permet de :
- Stocker les métadonnées du Model Registry dans une base de données locale SQLite.
- Stocker les artefacts (modèles, fichiers) en local.

### 1.3 - Initialisation du Tracking MLFlow de test

In [7]:
import os

# Ignorer Git
os.environ['GIT_PYTHON_REFRESH'] = 'quiet'

# Vérification si la variable d'environnement est bien définie
print(os.environ.get('GIT_PYTHON_REFRESH'))

quiet


In [8]:
# Définir le serveur de tracking (local ou distant) - ici local
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Démarrer une nouvelle expérimentation
mlflow.set_experiment("modele_test_quickstart_2")

MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=modele_test_quickstart_2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000226E92F2590>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

### 1.4 - Création d'un modèle pour tester l'initialisation

In [ ]:
# Charger les données Iris (pour la démonstration)
data = load_iris()
X = data.data
y = data.target

# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {
    "random_state": 42,
}

# Initialiser le modèle
model = LogisticRegression(**params)

# Entraînement du modèle
model.fit(X_train, y_train)
    
# Faire des prédictions sur le jeu de test
y_pred = model.predict(X_test)
#y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probabilités pour AUC binaire
y_pred_prob = model.predict_proba(X_test)  # Probabilités pour AUC

# Calculer accuracy et AUC
accuracy = accuracy_score(y_test, y_pred)
# auc = roc_auc_score(y_test, y_pred_prob) --> si le problème était binaire
auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='macro')

### 1.5 - Enregistrer le modèle et ses métadonnées dans MLFlow

Créons une fonction :

In [ ]:
def tracking_mlflow(tag, model, params, metrics, registered_model_name, X_train):

    # Démarrer une expérimentation dans MLflow
    with mlflow.start_run():
    
        # Log the hyperparameters
        mlflow.log_params(params)
        
        # Loguer les métriques
        for key, value in metrics.items():
            mlflow.log_metric(key, value)
            #mlflow.log_metric("accuracy", accuracy)
            #mlflow.log_metric("auc", auc)
    
        # Set a tag that we can use to remind ourselves what this run was for
        mlflow.set_tag(tag[0], tag[1])
    
        # Infer the model signature
        signature = infer_signature(X_train, model.predict(X_train))
        
        # Loguer le modèle
        model_info = mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="iris_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=registered_model_name,
        )

        for key, value in metrics.items():
            print(key, ":" , value)
    
    print(f"Modèle enregistré en version {model_info.mlflow_version}")

    return model_info

In [ ]:
metrics = {"accuracy": accuracy, "auc": auc}
registered_model_name="modele_test_quickstart_2"
tag = "Training Info", "Basic LR model for iris data"

model_info = tracking_mlflow(tag, model, params, metrics, registered_model_name, X_train)

**Lister les versions du modèle**

In [ ]:
client = mlflow.tracking.MlflowClient()
versions = client.get_latest_versions("modele_test_quickstart_2")
for v in versions:
    print(f"Version: {v.version}, Statut: {v.current_stage}")

### 1.6 - Chargez le modèle en tant que fonction Python, et l'utiliser pour une prédiction

In [ ]:
from sklearn import datasets

# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

### 1.7 - Lancement de l'UI

Pour  la visualisation et la comparaison des expérimentations, ainsi que le stockage de manière centralisée des modèles.

Cliquer sur ce lien : http://127.0.0.1:5000/

## 2 - Analyser le jeu de données

Je suis le kernel suivant : https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction/notebook

### 2.1 - Charger les données du fichier application_train

Le fichier application_test n'est pas utile, car il est destiné au concours Kaggle. Il ne contient pas la variable cible.

In [ ]:
# Training data
app_train = pd.read_csv('C:/Users/admin/Documents/Projets/Projet_7/data_projet/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

The training data has 307511 observations (each one a separate loan) and 122 features (variables) including the TARGET (the label we want to predict)

### 2.2 - Analyser la distribution de la variable cible TARGET

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist();

Il y a un déséquilibre important entre les 2 classes :
- 0 : le prêt a été remboursé
- 1 : le prêt n'a pas été remboursé.

Il y a plus de prêts remboursés que de prêts non remboursés.

### 2.3 - Examiner les valeurs manquantes

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(app_train)
missing_values.head(70)

### 2.4 - Examiner les types de colonnes

In [ ]:
# Number of each type of column
app_train.dtypes.value_counts()

Regardons le nombre de valeurs uniques dans les colonnes de type object:

In [ ]:
# Number of unique classes in each object column
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

La plupart des variables catégorielles ont un nombre relativement faible d'entrées uniques. Nous devons transformer ces catégories en variable numérique via un processus d'encodage : étiquettes (label) versus one-hot.

Le problème avec le codage des étiquettes est qu'il donne aux catégories un ordre arbitraire.

Nous utiliserons le codage d'étiquettes (Binary encode) pour toutes les variables catégorielles avec seulement 2 catégories, et le codage one-hot (get_dummies de pandas) pour toutes les variables catégorielles avec plus de 2 catégories.

### 2.6 - Recherche des valeurs aberrantes

In [ ]:
app_train.describe()

Les valeurs DAYS_xxx sont négatives car elles sont enregistrés par rapport à la demande de prêt en cours. Elles seront traitées dans le feature engineering.

**La valeur max deLa variable DAYS_EMPLOYED contient des valeurs aberrantes :**

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

Les anomalies ont un taux de défaut plus faible.

Puisque toutes les anomalies ont exactement la même valeur, nous voulons les remplir avec la même valeur au cas où tous ces prêts partageraient quelque chose en commun. En guise de solution, nous allons remplir les valeurs anormales avec un nombre (np.nan) lors du feature engineering.

### 2.7 - Corrélations

In [ ]:
# Find correlations with the target and sort
correlations = app_train.select_dtypes(include=['number']).corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [ ]:
# Find the correlation of the positive days since birth and target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

Il existe une relation linéaire négative avec la cible, ce qui signifie qu'à mesure que les clients vieillissent, ils ont tendance à rembourser leurs prêts à temps plus souvent.

Commençons par examiner cette variable. Tout d'abord, nous pouvons créer un histogramme de l'âge. Nous mettrons l'axe des x en années pour rendre le tracé un peu plus compréhensible.

In [ ]:
# Set the style of plots
plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

Pour visualiser l'effet de l'âge sur la cible, nous allons ensuite créer un graphique d'estimation de la densité du noyau (KDE) coloré par la valeur de la cible. 

Un graphique d'estimation de la densité du noyau montre la distribution d'une seule variable et peut être considéré comme un histogramme lissé (il est créé en calculant un noyau, généralement un gaussien, à chaque point de données, puis en faisant la moyenne de tous les noyaux individuels pour développer une seule courbe lisse). 

Nous utiliserons le kdeplot de Seaborn pour ce graphique.

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

La courbe cible == 1 penche vers l'extrémité la plus jeune de la fourchette. Bien que cette corrélation ne soit pas significative (coefficient de corrélation de -0,07), cette variable sera probablement utile dans un modèle d'apprentissage automatique car elle affecte la cible. Examinons cette relation d'une autre manière : le taux moyen de non-remboursement des prêts par tranche d'âge.

Pour réaliser ce graphique, nous découpons d'abord la catégorie d'âge en tranches de 5 ans chacune. Ensuite, pour chaque tranche, nous calculons la valeur moyenne de la cible, qui nous indique le ratio de prêts non remboursés dans chaque catégorie d'âge.

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

La tendance est claire : les jeunes demandeurs sont plus susceptibles de ne pas rembourser leur prêt ! Le taux de non-remboursement est supérieur à 10 % pour les trois tranches d’âge les plus jeunes et inférieur à 5 % pour la tranche d’âge la plus âgée.

Ces informations pourraient être directement utilisées par la banque : comme les jeunes clients sont moins susceptibles de rembourser leur prêt, il faudrait peut-être leur fournir davantage de conseils ou de conseils en matière de planification financière. Cela ne signifie pas que la banque doit faire preuve de discrimination à l’égard des jeunes clients, mais il serait judicieux de prendre des mesures de précaution pour les aider à payer à temps.

Sources extérieures

Les 3 variables ayant les corrélations négatives les plus fortes avec la cible sont EXT_SOURCE_1, EXT_SOURCE_2 et EXT_SOURCE_3. Selon la documentation, ces caractéristiques représentent un « score normalisé provenant d'une source de données externe ». Je ne sais pas exactement ce que cela signifie, mais il peut s'agir d'une sorte de notation de crédit cumulative réalisée à l'aide de nombreuses sources de données.

Examinons ces variables.

Tout d'abord, nous pouvons montrer les corrélations des caractéristiques EXT_SOURCE avec la cible et entre elles.

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize = (8, 6))

# Heatmap of correlations
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

Les trois caractéristiques EXT_SOURCE ont des corrélations négatives avec la cible, ce qui indique qu'à mesure que la valeur de EXT_SOURCE augmente, le client est plus susceptible de rembourser le prêt. Nous pouvons également voir que DAYS_BIRTH est positivement corrélé à EXT_SOURCE_1, ce qui indique que l'un des facteurs de ce score est peut-être l'âge du client.

Nous pouvons ensuite examiner la distribution de chacune de ces caractéristiques colorées par la valeur de la cible. Cela nous permettra de visualiser l'effet de cette variable sur la cible.

In [ ]:
plt.figure(figsize = (10, 12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    
    # create a new subplot for each source
    plt.subplot(3, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

EXT_SOURCE_3 affiche la plus grande différence entre les valeurs de la cible. Nous pouvons clairement voir que cette caractéristique a une certaine relation avec la probabilité qu'un demandeur rembourse un prêt. La relation n'est pas très forte (en fait, elles sont toutes considérées comme très faibles, mais ces variables seront toujours utiles à un modèle d'apprentissage automatique pour prédire si un demandeur remboursera ou non un prêt à temps.

## 3 - Feature engineering

Je m'appuie sur le Kernel : https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script

On ne reprend pas les transformations de fichier applaction_test qui ne nous concerne pas ici.

In [ ]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

path = "C:/Users/admin/Documents/Projets/Projet_7/data_projet/"

### 3.1 - Créer une fonction pour le one-hot encoding

Elle prend en entrée un dataframe, et renvoie le dataframe avec les variables de type Object transformée vie one-hot

In [ ]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

### 3.2 - Pré traiter le fichier application_train

On transforme les variables catégorielles :
- celles avec 2 valeurs en mode binaire
- Celles avec plus que 2 valeurs avec one-hot

Puis on remplace les valeurs aberrantes de la variable DAYS_EMPLOYED vu précédemment.

Puis on crée de nouvelles features en pourcentage

In [ ]:
# Preprocess application_train.csv and application_test.csv
def application_train(num_rows = None, nan_as_category = False, path = None):
    # Read data and merge
    df = pd.read_csv(path + 'application_train.csv', nrows= num_rows)
    print("Train samples: {}".format(len(df)))
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']        # proportion de la vie d'une personne passée en emploi
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']     # rapport entre le revenu total et le montant du crédit demandé
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS'] # revenu par membre du foyer
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']   # part du revenu utilisée pour rembourser une annuité
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']                # proportion du crédit remboursée à chaque échéance
    #gc.collect()
    return df

Testons ce pré traitement :

In [ ]:
df = application_train(num_rows=10000, path=path)
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

### 3.4 - Pré traiter les fichiers bureau et bureau_balance

Créons une fonction pour pré-traiter et agrèger les fichiers bureau.csv et bureau_balance.csv pour extraire des caractéristiques utiles.

- bureau.csv contient des informations sur les crédits passés des clients auprès d'autres banques.
- bureau_balance.csv contient l'historique mensuel des statuts des crédits enregistrés dans bureau.csv.

La fonction réalise les actions suivantes :
- Encodage des variables catégorielles (One-Hot Encoding)
- Agrégation de bureau_balance.csv par SK_ID_BUREAU
- Agrégation de bureau.csv par SK_ID_CURR
- Séparation entre crédits actifs et clôturés (Création des caractéristiques spécifiques aux crédits actifs et clôturés).

In [ ]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True, path = None):
    bureau = pd.read_csv(path + 'bureau.csv', nrows = num_rows)
    bb = pd.read_csv(path + 'bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}  # Date la plus ancienne, la plus récente d'un enregistrement, et le nbre d'enregistrements pour un crédit donné
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    #gc.collect()
    
    # Bureau and bureau_balance numeric features
    # agrège toutes les valeurs numériques de chaque client
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    # et agrège toutes les variables catégorielles en prenant la moyenne.
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})

    # les nouvelles colonnes auront le préfix BURO
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

    # Séparation entre crédits actifs et clôturés
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    #gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    #gc.collect()
    return bureau_agg

Testons avec ce pré traitement en plus :

In [ ]:
df = application_train(num_rows = 10000, path = path)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows = 10000, path = path)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')

df.shape

In [ ]:
df.head()

In [ ]:
bureau.head()

### 3.5 - Prétraiter le fichier previous_application

Créons une fonction pour prétraiter le fichier previous_application.csv, qui contient des informations sur les demandes de crédit précédentes des clients.

- Encodage One-Hot pour les colonnes catégorielles
- Remplacement des valeurs spécifiques par NaN
- Création de nouvelles caractéristiques (comme des pourcentages)
- Agrègation les données par SK_ID_CURR (identifiant client) à la fois pour des caractéristiques numériques et catégorielles
- Séparation des demandes approuvées et refusées, puis les agrège séparément

In [ ]:
# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True, path = None):
    prev = pd.read_csv(path + 'previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    #gc.collect()
    return prev_agg

Testons avec ce pré traitement avec les précédents :

In [ ]:
df = application_train(num_rows = 10000, path = path)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows = 10000, path = path)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')
with timer("Process previous_applications"):
    prev = previous_applications(num_rows = 10000, path = path)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')

df.shape

In [ ]:
prev.head()

### 3.6 - Prétraiter le fichier POS_CASH_balance

Créons une fonction pour rétraiter les données provenant du fichier POS_CASH_balance.csv, qui contient des informations sur l'historique des paiements effectués avec les cartes de crédit ou autres moyens de paiement utilisés par les clients.

- Encodage One-Hot des colonnes catégorielles.
- Agrégations statistiques sur des colonnes numériques spécifiques, telles que la valeur maximale, moyenne et la taille de certaines variables.
- Agrègation des informations des colonnes catégorielles avec la moyenne.
- Calcul du nombre de comptes POS par client.

In [ ]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True, path = None):
    pos = pd.read_csv(path + 'POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    #gc.collect()
    return pos_agg

Testons avec ce pré traitement avec les précédents :

In [ ]:
df = application_train(num_rows = 10000, path = path)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows = 10000, path = path)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal
with timer("Process previous_applications"):
    prev = previous_applications(num_rows = 10000, path = path)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process POS-CASH balance"):
    pos = pos_cash(num_rows = 10000, path = path)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal

df.shape

In [ ]:
pos.head()

In [ ]:
### 3.7 - Prétraiter le fichier installments_payments

Créons une fonction pour prétraiter le fichier contenant des informations sur les paiements de crédits ou de prêts par les clients.

- Encodage des variables catégorielles
- Calcul des nouvelles caractéristiques

In [ ]:
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True, path = path):
    ins = pd.read_csv(path + 'installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT'] # pourcentage de paiement par rapport à l'annuité
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT'] # Différence entre le paiement effectué et l'annuité
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT'] # Jours de retard (DPD - Days Past Due) et jours avant l'échéance (DBD - Days Before Due)
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT'] 
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0) # Correction des valeurs négatives 
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0) #Correction des valeurs négatives 
    
    # Features: Perform aggregations
    # Agrégation des caractéristiques numériques
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }

    # Agrégation des colonnes catégorielles
    for cat in cat_cols:
        aggregations[cat] = ['mean']

    # Application des agrégations par client (SK_ID_CURR)
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    #gc.collect()
    return ins_agg

Testons avec ce pré traitement avec les précédents :

In [ ]:
df = application_train(num_rows = 10000, path = path)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows = 10000, path = path)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal
with timer("Process previous_applications"):
    prev = previous_applications(num_rows = 10000, path = path)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process POS-CASH balance"):
    pos = pos_cash(num_rows = 10000, path = path)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process installments payments"):
    ins = installments_payments(num_rows = 10000, path = path)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')

df.shape

In [ ]:
ins.head()

### 3.8 - Prétraiter le fichier credit_card_balance

Créons une fonction pour prétraiter un fichier contenant des informations sur les cartes de crédit.

- Encodage des variables catégorielles
- Suppression de la colonne SK_ID_PREV
- Agrégation des données par SK_ID_CURR (ID client), en calculant des statistiques comme le minimum, maximum, moyenne, somme, et variance pour chaque colonne
- Ajout d'une colonne indiquant le nombre de cartes de crédit par client.

In [ ]:
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True, path = path):
    cc = pd.read_csv(path + 'credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    #gc.collect()
    return cc_agg

Testons avec ce pré traitement avec les précédents :

In [ ]:
df = application_train(num_rows = 10000, path = path)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows = 10000, path = path)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal
with timer("Process previous_applications"):
    prev = previous_applications(num_rows = 10000, path = path)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process POS-CASH balance"):
    pos = pos_cash(num_rows = 10000, path = path)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process installments payments"):
    ins = installments_payments(num_rows = 10000, path = path)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal
with timer("Process credit card balance"):
    cc = credit_card_balance(num_rows = 10000, path = path)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal

df.shape

In [ ]:
cc.head()

### 3.9 - créer le dataframe complet

In [ ]:
df = application_train(path = path)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(path = path)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal
with timer("Process previous_applications"):
    prev = previous_applications(path = path)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process POS-CASH balance"):
    pos = pos_cash(path = path)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')   # jointure à gauche avec le dataframe principal
with timer("Process installments payments"):
    ins = installments_payments(path = path)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal
with timer("Process credit card balance"):
    cc = credit_card_balance( path = path)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')  # jointure à gauche avec le dataframe principal

df.shape

### 3.10 - Traiter les valeurs manquantes

In [ ]:
# Missing values statistics
missing_values = missing_values_table(df)
missing_values.head(130)

## 4 - Créer un score métier pour l'entraînement des modèles

### 4.1 - Créer une fonction qui calcule le coût des erreurs de prédiction

In [ ]:
def calculate_errors_cost(y_true, y_pred):
    # Calcule le coût des erreurs de prédiction : 10*FN + FP"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()  # ravael pou transformer la matrice 2x2 en 4 valeurs
    return 10 * fn + fp

### 4.2 - Créer le score métier

In [ ]:
# Création du scorer métier - plus le score est petit mieux sait
business_score = make_scorer(business_cost, greater_is_better=False)

### 4.3 - Créer une fonction qui trouve le seuil optimal

Le seuil optimal est la valeur à partir de laquelle on décide si un client doit obtenir ou non un crédit, basé sur la probabilité prédite par le modèle. Le but est de trouver un seuil qui minimise le coût métier, c'est-à-dire le coût total des erreurs de prédiction en tenant compte des faux positifs (FP) et des faux négatifs (FN), où les faux négatifs sont 10 fois plus coûteux.

Le seuil optimal est celui qui donne le coût total le plus bas.

In [ ]:
def optimal_threshold(y_true, y_scores):
    # y_scores est la liste des probabilités d'être en classe 1
    # Trouve le seuil qui minimise le coût métier : 10*FN + FP
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)

    # tpr : Probabilité qu'un bon client (1) soit bien détecté comme bon
    # fpr : Probabilité qu'un mauvais client (0) soit faussement accepté comme bon

    # Calcul du coût métier pour chaque seuil =
    # Coûts des faux négatifs 10 * (1 - tpr[i]) * sum(y_true)
    # + Coûts total des faux positifs fpr[i] * (len(y_true) - sum(y_true))
    costs = [10 * (1 - tpr[i]) * sum(y_true) + fpr[i] * (len(y_true) - sum(y_true))
         for i in range(len(thresholds))]

    return thresholds[np.argmin(costs)]  # Retourne le seuil optimal

### 4.4 - Créer le score "seuil métier"

In [ ]:
def business_threshold_score(estimator, X, y):
    # Calcule le seuil optimal pour minimiser le coût métier
    y_scores = estimator.predict_proba(X)[:, 1]  # Probabilité d'être en classe 1
    return optimal_threshold(y, y_scores)  # Retourne le seuil optimal

# Création du scorer
scorer_threshold = make_scorer(business_threshold_score, needs_proba=True)

In [ ]:
DummyClassifier, Regression Logistique, Random Forest, Decision Tree, XGBoost, Lightgbm

Courbe Roc, AUC, f1-score, precision, recall, time

matrice de confusion